In [40]:
#Importing libraries
import numpy as np
import pandas as pd
from datetime import datetime

In [41]:
#loading data and printing the first 5 rows
data=pd.read_csv('train.csv')
data.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


# A) Look for the missing values in all the columns and either impute them (replace with mean,median, or mode) or drop them. Justify your action for this task.

In [42]:
print("Percentage of missing values in the each column:")
for column in data.columns:
    missing_percentage = data[column].isnull().sum() / len(data) * 100
    print(f"{column}: {missing_percentage:.2f}%")

Percentage of missing values in the each column:
Unnamed: 0: 0.00%
Name: 0.00%
Location: 0.00%
Year: 0.00%
Kilometers_Driven: 0.00%
Fuel_Type: 0.00%
Transmission: 0.00%
Owner_Type: 0.00%
Mileage: 0.03%
Engine: 0.62%
Power: 0.62%
Seats: 0.65%
New_Price: 86.06%
Price: 0.00%


Here you can see few null values percentage in Mileage, Engine, Power, Seats, New_price.

In [43]:
# Define functions for cleaning numeric columns and performing imputation
def clean_and_convert(column):
    if column in data.columns:
        # Remove non-numeric characters and convert to float
        data[column] = data[column].replace('[^0-9.]', '', regex=True).astype(float)

def impute_mean(column):
    if column in data.columns:
        data[column] = data[column].fillna(data[column].mean())

# Clean and convert columns with units to numeric format
for col in ['Mileage', 'Engine', 'Power', 'Seats']:
    clean_and_convert(col)
    impute_mean(col)

# Drop the 'New_Price' column if it exists
if 'New_Price' in data.columns:
    data = data.drop('New_Price', axis=1)

# Check final missing values
print("Missing values after processing:")
print(data.isnull().sum())


Missing values after processing:
Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64


Explanation:

Mean Imputation: Columns like Mileage, Engine, Power, and Seats have less than 1% missing values. Using the mean to fill in these gaps is effective and unlikely to skew the data significantly, thus preserving overall data integrity.

Dropping Column (New_Price): The New_Price column has over 86% missing values, making it largely incomplete. Due to its high level of missing data, it's not useful for analysis and is therefore removed from the dataset.

#B) Remove the units from some of the attributes and only keep the numerical values (for example remove kmpl from “Mileage”, CC from “Engine”, bhp from “Power”, and lakh from
“New_price”). (4 points)

In [44]:
# Function to clean units and keep only numeric values
def clean_units(df, column):
    df[column] = df[column].astype(str).str.extract(r'([\d.]+)').astype(float)

# Apply the cleaning function to the specified columns
columns_to_clean = ['Mileage', 'Engine', 'Power']
for col in columns_to_clean:
    clean_units(data, col)

# Verify the result
print(data[columns_to_clean].head())

   Mileage  Engine   Power
0    19.67  1582.0  126.20
1    13.00  1199.0   88.70
2    20.77  1248.0   88.76
3    15.20  1968.0  140.80
4    23.08  1461.0   63.10


#C) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value. (4 points).

In [45]:
# Apply one-hot encoding to the 'Fuel_Type' and 'Transmission' columns
data = pd.get_dummies(data, columns=['Fuel_Type', 'Transmission'], drop_first=False)

print(data.head())

   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  Price  \
0              41000      First    19.67  1582.0  126.20    5.0  12.50   
1              46000      First    13.00  1199.0   88.70    5.0   4.50   
2              87000      First    20.77  1248.0   88.76    7.0   6.00   
3              40670     Second    15.20  1968.0  140.80    5.0  17.74   
4              86999      First    23.08  1461.0   63.10    5.0   3.50   

   Fuel_Type_Diesel  Fuel_Type_Electric  Fuel_Type_Petrol  \
0              True               False             False   
1             Fals

#D)Create one more feature and add this column to the dataset (you can use mutate function in R for this). For example, you can calculate the current age of the car by subtracting “Year” value from the current year. (4 points)

In [46]:
# Get the current year
current_year = datetime.now().year

# Calculate the age of the car and add it as a new column
data['Car_Age'] = current_year - data['Year']

# Result
print(data[['Year', 'Car_Age']].head())

   Year  Car_Age
0  2015       10
1  2011       14
2  2012       13
3  2013       12
4  2013       12


#E) Perform select, filter, rename, mutate, arrange and summarize with group by operations (or their equivalent operations in python) on this dataset.

In [47]:
# Select only the columns 'Name', 'Year', 'Kilometers_Driven', and 'Price'
selected_data = data[['Name', 'Year', 'Kilometers_Driven', 'Price']]
print("Selected Columns:\n", selected_data.head())

Selected Columns:
                                Name  Year  Kilometers_Driven  Price
0  Hyundai Creta 1.6 CRDi SX Option  2015              41000  12.50
1                      Honda Jazz V  2011              46000   4.50
2                 Maruti Ertiga VDI  2012              87000   6.00
3   Audi A4 New 2.0 TDI Multitronic  2013              40670  17.74
4            Nissan Micra Diesel XV  2013              86999   3.50


In [48]:
# Filter rows where Price is greater than 5 lakhs and the car is Diesel
filtered_data = data[(data['Price'] > 5) & (data['Fuel_Type_Diesel'] == 1)]

print("Filtered Data:\n", filtered_data.head())

Filtered Data:
    Unnamed: 0                                 Name    Location  Year  \
0           1     Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
2           3                    Maruti Ertiga VDI     Chennai  2012   
3           4      Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
5           7    Toyota Innova Crysta 2.8 GX AT 8S      Mumbai  2016   
6           8  Volkswagen Vento Diesel Comfortline        Pune  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  Price  \
0              41000      First    19.67  1582.0  126.20    5.0  12.50   
2              87000      First    20.77  1248.0   88.76    7.0   6.00   
3              40670     Second    15.20  1968.0  140.80    5.0  17.74   
5              36000      First    11.36  2755.0  171.50    8.0  17.50   
6              64430      First    20.54  1598.0  103.60    5.0   5.20   

   Fuel_Type_Diesel  Fuel_Type_Electric  Fuel_Type_Petrol  \
0              True               False      

In [49]:
# Rename the column 'Kilometers_Driven' to 'KM_Driven'
renamed_data = data.rename(columns={'Kilometers_Driven': 'KM_Driven'})
print("Renamed Data:\n", renamed_data.head())

Renamed Data:
    Unnamed: 0                              Name    Location  Year  KM_Driven  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015      41000   
1           2                      Honda Jazz V     Chennai  2011      46000   
2           3                 Maruti Ertiga VDI     Chennai  2012      87000   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013      40670   
4           6            Nissan Micra Diesel XV      Jaipur  2013      86999   

  Owner_Type  Mileage  Engine   Power  Seats  Price  Fuel_Type_Diesel  \
0      First    19.67  1582.0  126.20    5.0  12.50              True   
1      First    13.00  1199.0   88.70    5.0   4.50             False   
2      First    20.77  1248.0   88.76    7.0   6.00              True   
3     Second    15.20  1968.0  140.80    5.0  17.74              True   
4      First    23.08  1461.0   63.10    5.0   3.50              True   

   Fuel_Type_Electric  Fuel_Type_Petrol  Transmission_Automatic  

In [50]:
# Create a new column 'Car_Age' based on the 'Year' column
data['Car_Age'] = current_year - data['Year']
print("Data with Car_Age:\n", data[['Year', 'Car_Age']].head())

Data with Car_Age:
    Year  Car_Age
0  2015       10
1  2011       14
2  2012       13
3  2013       12
4  2013       12


In R, arrange() is used for sorting. In pandas, you can use sort_values().

In [51]:
# Sort the data by 'Price' in descending order
arranged_data = data.sort_values(by='Price', ascending=False)
print("Arranged Data (sorted by Price):\n", arranged_data.head())

Arranged Data (sorted by Price):
       Unnamed: 0                                         Name    Location  \
3952        4079  Land Rover Range Rover 3.0 Diesel LWB Vogue   Hyderabad   
5620        5781                   Lamborghini Gallardo Coupe       Delhi   
5752        5919                       Jaguar F Type 5.0 V8 S   Hyderabad   
1457        1505              Land Rover Range Rover Sport SE       Kochi   
1917        1974                           BMW 7 Series 740Li  Coimbatore   

      Year  Kilometers_Driven Owner_Type  Mileage  Engine  Power  Seats  \
3952  2017              25000      First    13.33  2993.0  255.0    5.0   
5620  2011               6500      Third     6.40  5204.0  560.0    2.0   
5752  2015               8000      First    12.50  5000.0  488.1    2.0   
1457  2019              26013      First    12.65  2993.0  255.0    5.0   
1917  2018              28060      First    12.05  2979.0  320.0    5.0   

       Price  Fuel_Type_Diesel  Fuel_Type_Electric  

In R, group_by() and summarize() are used together to aggregate data. In Pandas, you can use groupby() with aggregation functions like mean().

In [52]:
# Group by 'Fuel_Type' and calculate the average 'Price' and total 'Kilometers_Driven'
summary_data = data.groupby('Fuel_Type_Diesel').agg({
    'Price': 'mean',
    'Kilometers_Driven': 'sum'
}).reset_index()

# Rename aggregated columns for clarity
summary_data = summary_data.rename(columns={'Price': 'Avg_Price', 'Kilometers_Driven': 'Total_KM_Driven'})
print("Summarized Data (grouped by Fuel_Type):\n", summary_data)

Summarized Data (grouped by Fuel_Type):
    Fuel_Type_Diesel  Avg_Price  Total_KM_Driven
0             False   5.761988        128269656
1              True  12.960686        213254401
